In [1]:
# Lab 12 Character Sequence RNN
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)  # reproducibility

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}


In [25]:
char_dic

{' ': 20,
 "'": 7,
 ',': 18,
 '.': 23,
 'a': 24,
 'b': 0,
 'c': 2,
 'd': 9,
 'e': 22,
 'f': 17,
 'g': 4,
 'h': 5,
 'i': 6,
 'k': 13,
 'l': 19,
 'm': 16,
 'n': 11,
 'o': 10,
 'p': 3,
 'r': 8,
 's': 12,
 't': 21,
 'u': 1,
 'w': 14,
 'y': 15}

In [2]:
# hyperparameters
learning_rate = 0.1
num_epochs = 500
input_size = len(char_set)  # RNN input size (one hot size)
hidden_size = len(char_set)  # RNN output size
num_classes = len(char_set)  # final output size (RNN or softmax, etc.)
sequence_length = 10  # any arbitrary number
num_layers = 2  # number of layers in RNN

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

x_data = torch.Tensor(dataX)
y_data = torch.LongTensor(dataY)


0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [3]:

# one hot encoding


def one_hot(x, num_classes):
    idx = x.long()
    idx = idx.view(-1, 1)
    x_one_hot = torch.zeros(x.size()[0] * x.size()[1], num_classes)
    x_one_hot.scatter_(1, idx, 1)
    x_one_hot = x_one_hot.view(x.size()[0], x.size()[1], num_classes)
    return x_one_hot


x_one_hot = one_hot(x_data, num_classes)

inputs = Variable(x_one_hot)
labels = Variable(y_data)


In [4]:
x_one_hot


( 0 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
   0   0   0  ...    0   0   0

( 1 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   1   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0

( 2 ,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
... 

(167,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   1
   0   0   0  ...    0   0   0
   0   0   0  ...    1   0   0

(168,.,.) = 
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0

In [5]:

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        # Set parameters for RNN block
        # Note: batch_first=False by default.
        # When true, inputs are (batch_size, sequence_length, input_dimension)
        # instead of (sequence_length, batch_size, input_dimension)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers,batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        # h_0 = Variable(torch.zeros(
        # self.num_layers, x.size(0), self.hidden_size))
        # c_0 = Variable(torch.zeros(
        # self.num_layers, x.size(0), self.hidden_size))

        # Propagate input through LSTM
        # Input: (batch, seq_len, input_size)
        out, _ = self.lstm(x, (h_0, c_0))
        # Note: the output tensor of LSTM in this case is a block with holes
        # > add .contiguous() to apply view()
        out = out.contiguous().view(-1, self.hidden_size)
        # Return outputs applied to fully connected layer
        out = self.fc(out)
        return out


In [6]:

# Instantiate RNN model
lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

# Set loss and optimizer function
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)


In [7]:

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(inputs)
    optimizer.zero_grad()
    # obtain the loss function
    # flatten target labels to match output
    loss = criterion(outputs, labels.view(-1))
    loss.backward()
    optimizer.step()
    # obtain the predicted indices of the next character
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    idx = idx.reshape(-1, sequence_length)  # (170,10)
    # display the prediction of the last sequence
    result_str = [char_set[c] for c in idx[-2]]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")


epoch: 1, loss: 3.241
Predicted string:  ..........
epoch: 2, loss: 3.006
Predicted string:  ddddddddss
epoch: 3, loss: 2.926
Predicted string:  e         
epoch: 4, loss: 2.965
Predicted string:            
epoch: 5, loss: 2.853
Predicted string:            
epoch: 6, loss: 2.849
Predicted string:            
epoch: 7, loss: 2.837
Predicted string:            
epoch: 8, loss: 2.815
Predicted string:            
epoch: 9, loss: 2.791
Predicted string:            
epoch: 10, loss: 2.759
Predicted string:   o  oo  o 
epoch: 11, loss: 2.708
Predicted string:  oo  oo  o 
epoch: 12, loss: 2.635
Predicted string:  oo  oo  o 
epoch: 13, loss: 2.569
Predicted string:  oo  oo  o 
epoch: 14, loss: 2.504
Predicted string:  oo  oo  o 
epoch: 15, loss: 2.438
Predicted string:  oo  oo oo 
epoch: 16, loss: 2.361
Predicted string:  on  oo  o 
epoch: 17, loss: 2.249
Predicted string:  on  oo tnn
epoch: 18, loss: 2.176
Predicted string:  on  oerann
epoch: 19, loss: 2.135
Predicted string:  en  oe tnm
ep

epoch: 155, loss: 0.232
Predicted string:  tf the sea
epoch: 156, loss: 0.232
Predicted string:  tf the sea
epoch: 157, loss: 0.232
Predicted string:  tf the sea
epoch: 158, loss: 0.232
Predicted string:  tf the sea
epoch: 159, loss: 0.232
Predicted string:  tf the sea
epoch: 160, loss: 0.232
Predicted string:  tf the sea
epoch: 161, loss: 0.232
Predicted string:  tf the sea
epoch: 162, loss: 0.232
Predicted string:  tf the sea
epoch: 163, loss: 0.232
Predicted string:  tf the sea
epoch: 164, loss: 0.232
Predicted string:  tf the sea
epoch: 165, loss: 0.233
Predicted string:  tf the sea
epoch: 166, loss: 0.234
Predicted string:  tf the sea
epoch: 167, loss: 0.232
Predicted string:  tf the sea
epoch: 168, loss: 0.232
Predicted string:  tf the sea
epoch: 169, loss: 0.232
Predicted string:  tf the sea
epoch: 170, loss: 0.232
Predicted string:  tf the sea
epoch: 171, loss: 0.232
Predicted string:  tf the sea
epoch: 172, loss: 0.232
Predicted string:  tf the sea
epoch: 173, loss: 0.232
Pred

epoch: 308, loss: 0.230
Predicted string:  tf the sea
epoch: 309, loss: 0.230
Predicted string:  tf the sea
epoch: 310, loss: 0.230
Predicted string:  tf the sea
epoch: 311, loss: 0.230
Predicted string:  tf the sea
epoch: 312, loss: 0.230
Predicted string:  tf the sea
epoch: 313, loss: 0.230
Predicted string:  tf the sea
epoch: 314, loss: 0.230
Predicted string:  tf the sea
epoch: 315, loss: 0.230
Predicted string:  tf the sea
epoch: 316, loss: 0.230
Predicted string:  tf the sea
epoch: 317, loss: 0.230
Predicted string:  tf the sea
epoch: 318, loss: 0.230
Predicted string:  tf the sea
epoch: 319, loss: 0.230
Predicted string:  tf the sea
epoch: 320, loss: 0.230
Predicted string:  tf the sea
epoch: 321, loss: 0.230
Predicted string:  tf the sea
epoch: 322, loss: 0.230
Predicted string:  tf the sea
epoch: 323, loss: 0.230
Predicted string:  tf the sea
epoch: 324, loss: 0.230
Predicted string:  tf the sea
epoch: 325, loss: 0.230
Predicted string:  tf the sea
epoch: 326, loss: 0.230
Pred

epoch: 460, loss: 0.229
Predicted string:  tf the sea
epoch: 461, loss: 0.229
Predicted string:  tf the sea
epoch: 462, loss: 0.229
Predicted string:  tf the sea
epoch: 463, loss: 0.229
Predicted string:  tf the sea
epoch: 464, loss: 0.229
Predicted string:  tf the sea
epoch: 465, loss: 0.229
Predicted string:  tf the sea
epoch: 466, loss: 0.229
Predicted string:  tf the sea
epoch: 467, loss: 0.229
Predicted string:  tf the sea
epoch: 468, loss: 0.229
Predicted string:  tf the sea
epoch: 469, loss: 0.229
Predicted string:  tf the sea
epoch: 470, loss: 0.229
Predicted string:  tf the sea
epoch: 471, loss: 0.229
Predicted string:  tf the sea
epoch: 472, loss: 0.229
Predicted string:  tf the sea
epoch: 473, loss: 0.229
Predicted string:  tf the sea
epoch: 474, loss: 0.229
Predicted string:  tf the sea
epoch: 475, loss: 0.229
Predicted string:  tf the sea
epoch: 476, loss: 0.229
Predicted string:  tf the sea
epoch: 477, loss: 0.229
Predicted string:  tf the sea
epoch: 478, loss: 0.229
Pred